## Plotly and Dashboards

* Plotly
* Creating Dashboards

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df = pd.read_csv('data/snow-alta-1990-2017.csv')

## Clean Data

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
def tweak_alta(df):
    return (df
           .assign(DATE=pd.to_datetime(df.DATE).dt.tz_localize('America/Denver'))
           .loc[: ,['DATE', 'STATION', 'NAME', 'LATITUDE', 'LONGITUDE',
                   'PRCP', 'SNOW', 'SNWD', 'TMIN', 'TMAX', 'TOBS'
                  ]]
            .assign(MONTH=lambda df_: df_.DATE.dt.month,
                    YEAR=lambda df_: df_.DATE.dt.year,
                    SEASON=lambda df_: np.select([df_.MONTH < 5,
                                                 df_.MONTH > 10],
                [(df_.YEAR - 1).astype(str) + '-' + 
                 (df_.YEAR).astype(str) + ' Season',
                (df_.YEAR).astype(str) + '-' + 
                 (df_.YEAR + 1).astype(str) + ' Season'],
                default='Off Season'))
           )
        

tweak_alta(df)

In [ ]:
alta = tweak_alta(df)

In [ ]:
alta

## Plotting with Plotly

In [ ]:
import plotly
import plotly.express as px
pd.options.plotting.backend = 'plotly'

In [ ]:
(alta
.plot(x="DATE", y='SNWD'))

In [ ]:
(alta
 .query('SEASON.str.contains("2010-2011")')
 .plot(x='DATE', y='SNWD', title='2011 Season Snow Depth')
)

## Bar Plot

In [ ]:
(alta
 .SNOW
 .pipe(pd.cut, bins=10)
 .value_counts()
 #.plot.bar()
 .index
)

In [ ]:
(alta
 .SNOW
 .pipe(pd.cut, bins=10)
 .value_counts()
 .reset_index()
 .rename(columns={'SNOW': 'Bin (in)', 'count': 'Count'})
 .astype({'Bin (in)': str})
 .plot.bar(x='Bin (in)', y='Count', title='SNOW FALL COUNTS')
)

In [ ]:
(alta
 .SNOW
# .pipe(pd.cut, bins=10)
 .pipe(pd.cut, bins=[-1, 0, 1, 5, 10, 15, 20, 25, 40]) 
 .value_counts()
 .reset_index()
 .rename(columns={'SNOW': 'Bin (in)', 'count': 'Count'})
 .astype({'Bin (in)': str})
 .plot.bar(x='Bin (in)', y='Count', title='SNOW FALL COUNTS')
)

In [ ]:
(alta
 .query('SNOW > 0') 
 .SNOW
 .pipe(pd.cut, bins=[-1, 0, 1, 5, 10, 15, 20, 25, 40]) 
 .value_counts()
 .reset_index()
 .rename(columns={'SNOW': 'Bin (in)', 'count': 'Count'})
 .astype({'Bin (in)': str})
 .plot.bar(x='Bin (in)', y='Count', title='SNOW FALL COUNTS')
)

## Scatter Plot

In [ ]:
(alta
 .plot.scatter(x='MONTH', y='SNWD', c='TOBS', opacity=.2)
)

In [ ]:
(alta
 .plot.scatter(x='TOBS', y='SNWD', color='MONTH')
)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(
    x=alta.TOBS,
    y=alta.SNWD,
    mode='markers',
    marker=dict(
        size=10,
        color=alta.MONTH,  
        colorscale='Twilight',  
        colorbar=dict(title='Month') 
    )
))

fig

## Dash

In [ ]:
%%writefile dashdemo.py
import dash
from dash import dcc, html, Dash, Input, Output
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
pd.options.plotting.backend = 'plotly'

def tweak_alta(df):
    return (df
           .assign(DATE=pd.to_datetime(df.DATE).dt.tz_localize('America/Denver'))
           .loc[: ,['DATE', 'STATION', 'NAME', 'LATITUDE', 'LONGITUDE',
                   'PRCP', 'SNOW', 'SNWD', 'TMIN', 'TMAX', 'TOBS'
                  ]]
            .assign(MONTH=lambda df_: df_.DATE.dt.month,
                    YEAR=lambda df_: df_.DATE.dt.year,
                    SEASON=lambda df_: np.select([df_.MONTH < 5,
                                                 df_.MONTH > 10],
                [(df_.YEAR - 1).astype(str) + '-' + 
                 (df_.YEAR).astype(str) + ' Season',
                (df_.YEAR).astype(str) + '-' + 
                 (df_.YEAR + 1).astype(str) + ' Season'],
                default='Off Season'))
           )
        
df = pd.read_csv('data/snow-alta-1990-2017.csv')
alta = tweak_alta(df)
app = Dash(__name__)
fig = (alta
 .query('SEASON.str.contains("2010-2011")')
 .plot(x='DATE', y='SNWD', title='2011 Season Snow Depth')
)
app.layout = dash.html.Div(children=[
    dash.html.H1('Alta 2011 Season'),
    dash.dcc.Markdown('''## Line Plot of Snow Depth

* This is Markdown text.
* Plot of Snow Depth

    '''),
    dash.dcc.Graph(id='line-graph', figure=fig)
])

if __name__ == '__main__':
    app.run_server(debug=True)


## Widget

In [ ]:
%%writefile dashdemo2.py
import dash
from dash import dcc, html, Dash, Input, Output
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
pd.options.plotting.backend = 'plotly'

def tweak_alta(df):
    return (df
           .assign(DATE=pd.to_datetime(df.DATE).dt.tz_localize('America/Denver'))
           .loc[: ,['DATE', 'STATION', 'NAME', 'LATITUDE', 'LONGITUDE',
                   'PRCP', 'SNOW', 'SNWD', 'TMIN', 'TMAX', 'TOBS'
                  ]]
            .assign(MONTH=lambda df_: df_.DATE.dt.month,
                    YEAR=lambda df_: df_.DATE.dt.year,
                    SEASON=lambda df_: np.select([df_.MONTH < 5,
                                                 df_.MONTH > 10],
                [(df_.YEAR - 1).astype(str) + '-' + 
                 (df_.YEAR).astype(str) + ' Season',
                (df_.YEAR).astype(str) + '-' + 
                 (df_.YEAR + 1).astype(str) + ' Season'],
                default='Off Season'))
           )
        
df = pd.read_csv('data/snow-alta-1990-2017.csv')
alta = tweak_alta(df)
app = Dash(__name__)
fig = (alta
 .query('SEASON.str.contains("2010-2011")')
 .plot(x='DATE', y='SNWD', title='2011 Season Snow Depth')
)


app.layout = dash.html.Div(children=[
    dash.html.H1('Alta Season Snow Depths'),
    dash.dcc.Markdown('''## Line Plot of Snow Depth
* This is Markdown text.
* Plot of Snow Depth
    '''),
    dash.dcc.Dropdown(
                    options=alta.SEASON.unique(),
                    id='years'),
    dash.dcc.Graph(id='line-graph', figure=fig)
])
    
@app.callback(
    dash.Output(component_id='line-graph', component_property='figure'),
    dash.Input(component_id='years', component_property='value'),
)
def update_line_figure(max_year):
    return (alta
     .query(f'SEASON == @max_year')
     .plot(x='DATE', y='SNWD',
          title=f'{max_year} Snow Depth')
    )   

if __name__ == '__main__':
    app.run_server(debug=True)
